In [137]:
import numpy as np
import pandas as pd
import pandas_profiling
import math
import re

In [138]:
def is_number(s):
    """ Returns True is string is a number. """
    try:
        float(s)
        return True
    except ValueError:
        return False

In [139]:
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

### Data Cleaning

In [140]:
def is_job_status_and_living_area_reversed(x):
    return ((x["job_status"] != None and x["job_status"] in ("r", "c", "city", "remote", "remotee"))
             or (x["living_area"] != None and x["living_area"] in ("private_sector", "business_owner")))
def process_job_status(x):
    if x == None or x in ("nan", 'null', "", 'n.a'):
        return None
    elif x in ("private sector", "privattte", "private", "private_sector"):
        return "private_sector"
    elif x in ("government", "govt."):
        return "government"
    elif x in ("business_owner", "business owner", "biz"):
        return "business_owner"
    elif x in ("parental_leave", "parental leave"):
        return "parental_leave"
    else:
        return x
    
def process_living_area(x):
    if x == None or x in ("nan", 'null', "", 'n.a'):
        return None
    elif x == 'c':
        return 'city'
    elif x in ('r', 'remotee'):
        return 'remote'
    else:
        return x

def split_job_status_and_living_area(x):
    pair = x.lower().split("?") if x != None else [x, x]
    if len(pair) < 2:
        pair = [pair[0], None]
    return pair

def process_job_status_and_living_area(df):
    df["job_status"] = df["job_status and living_area"].astype(str).apply(split_job_status_and_living_area).apply(lambda x: x[0])
    df["living_area"] = df["job_status and living_area"].astype(str).apply(split_job_status_and_living_area).apply(lambda x: x[1])
    job_status = df.apply(lambda x: x["living_area"] if is_job_status_and_living_area_reversed(x) else x["job_status"], 1)
    living_area = df.apply(lambda x: x["job_status"] if is_job_status_and_living_area_reversed(x) else x["living_area"], 1)
    df["job_status"] = job_status.apply(lambda x: process_job_status(x))
    df["living_area"] = living_area.apply(lambda x: process_living_area(x))
    df.drop(columns='job_status and living_area',inplace=True)

In [141]:
process_job_status_and_living_area(train)
process_job_status_and_living_area(test)

In [142]:
def process_smoker_status(x):
    if x == None:
        return None
    elif x.startswith("non"):
        return "non-smoker"
    elif x.startswith("quit"):
        return "quit"
    elif x.startswith("active"):
        return "active_smoker"
    else:
        return None
train["smoker_status"] = train["smoker_status"].astype(str).apply(process_smoker_status)
test["smoker_status"] = test["smoker_status"].astype(str).apply(process_smoker_status)

Convert BMI to numeric

In [143]:
train["BMI"] = pd.to_numeric(train["BMI"],errors="coerce")

Treat '.,' as Missing Values

In [144]:
def replace_invalid_binary_values(df,columns):
    for col in columns:
        df[~df[col].isin(["0","1"])] = None
replace_invalid_binary_values(train,["high_BP","stroke_in_2018","heart_condition_detected_2017","married"])
replace_invalid_binary_values(train,["high_BP","heart_condition_detected_2017","married"])

Clean Sex and Age

In [145]:
def clean_sex_age(sex_age_list):
    if type(sex_age_list) is not list:
        return [None,None]
    # Strip and Upper case both sex and age
    sex_age_list[0],sex_age_list[1] = sex_age_list[0].strip().upper(), sex_age_list[1].strip().upper()
    
    # 2nd : first one is empty and second one is not numeric
    if (is_number(sex_age_list[0]) or (not sex_age_list[0] and not is_number(sex_age_list[1]))): 
        sex_age_list = sex_age_list[::-1]
    sex = sex_age_list[0].strip().upper()
    
    # Take
    if sex in ('FEMALE','FEMALLE'):
        sex = 'F'
    if sex in ('MALE','MMALE','MM'):
        sex = 'M'

    sex_age_list[0] = sex
    sex_age_list[1] = sex_age_list[1]
    return sex_age_list
def process_sex_age(df):
    df["sex_age_list"] = df["sex and age"].str.split(",").apply(clean_sex_age)
    df[['sex','age']] = pd.DataFrame(df["sex_age_list"].values.tolist(), index= df.index)
    df["age"] = pd.to_numeric(df["age"],errors='coerce').round()
    return df.drop(columns=["sex_age_list","sex and age"])

In [146]:
train = process_sex_age(train)

In [149]:
train.drop_duplicates(inplace=True)

In [ ]:
profile = train.profile_report(title='Medical Record Profiling Report')
profile.to_file(output_file="train_data_summary.html")

E:\Program\Anaconda\envs\NLP\lib\site-packages\pandas_profiling\model\correlations.py:34: RuntimeWarning: invalid value encountered in double_scalars
  return np.sqrt(phi2corr / min((kcorr - 1.0), (rcorr - 1.0)))
